# Supplemental explanations

## For Section 3: Evaluating Feature Contributions

The following explanations are adapted from the paper: Palczewska, A., Palczewski, J., Robinson, R. M., & Neagu, D. (2014). Interpreting random forest classification models using a feature contribution method. In *Integration of reusable systems* (pp. 193-218). Springer, Cham.

### Probabalistic Interpretation of Classification Random Forest Models

This is a probabalistic interpretation of the forest prediction process. Rather than thinking of the decisions as decisions, we'll think about them as probabilities of class membership. 

The classes in our model can be represented by the set $C = \{C_{1},C_{2},...,C_{K}\}$, where each $C$ is a class, and there are $K$ number of classes. Additionally, we can define the set $\Delta_{K}$ by the following: 

$$\Delta_{K} = \{(p_{1},...,p_{K}) \textrm{ } | \sum\limits_{k=1}^K p_{k} = 1 \textrm{ and } p_{k}\}$$

which can be stated in plain english as, "the set $p_{1}$ to $p_{K}$, such that the sum of all the $p$'s is 1, and all $p$ are greater than or equal to zero". There is one $p$ for each class. Taken together, all the elements in $\Delta_{K}$ form a probability distribution -- that's why they have to sum to 1. But what are the $p$'s? <br> 

For a given class $k$, we will define $\hat{Y}_{i,t}$ as a prediction of a tree, $t$, for an instance, $i$. This prediction specifically is that the tree has predicted that $i$ is of class $k$, so let's write $\hat{Y}_{(k)i,t}$ to clarify. $\hat{Y}_{(k)i,t}$ represents the probability that a tree in the forest will classify $i$ as being of class $k$. There are $T$ number of these $\hat{Y}_{(k)i,t}$'s, because there are T number of trees in the forest. Therefore, if we want to know the probability that the whole forest will predict $i$ to be class $k$, we do the following: 

$$\hat{Y}_{(k)i} = \frac{1}{T} \sum\limits_{t=1}^T \hat{Y}_{(k)i,t}$$

We do this for all the classes, which yields the set $\{\hat{Y}_{(1)i}, ... , \hat{Y}_{(k)i}\}$, which is the probability distribution over all classes: AKA, this is $\Delta_{K}$.

So to sum up: we represent the "decision" that each tree makes about an instance, as a probability of class membership. We sum up these probabilities for all trees for each class, which at the end yields a probability distribution for the prediction of that instance. 

**NOTE TO SELF:** *figure out what this means/if my interpretation of Y is wrong
"Consequently, the set of probability distributions on C is equivalent to the
probability weight assigned to class C1."*

### The Unanimity Condition: Using feature contributions to retrieve model predictions

### Feature contributions for multiclass models